In [ ]:
1

In [ ]:
import numpy as np
import pandas as pd

pd.options.display.float_format = lambda v: "{:_.2f}".format(v).replace("_", " ")

In [ ]:
from openfisca_france import CountryTaxBenefitSystem
from openfisca_core.parameters.parameter_scale import ParameterScale
from openfisca_core.reforms import Reform

In [ ]:
from sports import compute, fields, get_df, get_results, build_data

In [ ]:
from scenario import (
    QfFiscalReform,
    StatutReform,
    CEReform,
    gristSimulationReform,
    period,
)

In [ ]:
import sys

sys.path.append("../technique")
from utils import *

In [ ]:
base = CountryTaxBenefitSystem()
base.load_extension("openfisca_france_local")

In [ ]:
StatutReform(base)

In [ ]:
for f in fields:
    bareme = fields[f]["bareme"]
    if bareme:
        p = bareme(base.parameters)
        assert type(p) == ParameterScale

In [ ]:
from scenario import (
    QfFiscalReform,
    StatutReform,
    CEReform,
    gristSimulationReform,
    period,
    new_bracket,
)

In [ ]:
qf_fiscal_reform = QfFiscalReform(base)
statuts_reform = StatutReform(qf_fiscal_reform)
ce_reform = CEReform(statuts_reform)
reform = ce_reform

In [ ]:
(recap, dfs) = get_results(base, 1, reform)

In [ ]:
recap

In [ ]:
for n, df in dfs:
    print((n, df.shape))

In [ ]:
sum([df.shape[0] for (n, df) in dfs])

In [ ]:
dfs[0][1]

In [ ]:
full = recap
if reform:
    full[("Écart", "euros")] = (
        full[(("Recettes réforme", "Moyenne"))] - full[(("Recettes base", "Moyenne"))]
    )
    full[("Écart", "%")] = (
        full[(("Écart", "euros"))] / full[(("Recettes base", "Moyenne"))]
    )
    resume = (
        full.groupby(("Direction", ""))
        .sum(numeric_only=True)
        .sort_values(("Recettes base", "Moyenne"), ascending=False)
    )
    resume[("Écart", "%")] = (
        resume[(("Écart", "euros"))] / resume[(("Recettes base", "Moyenne"))]
    )
    resume

In [ ]:
resume[
    [
        ("Recettes base", "Moyenne"),
        ("Recettes réforme", "Moyenne"),
        ("Écart", "euros"),
        ("Écart", "%"),
    ]
]

In [ ]:
full[
    [
        ("Service", ""),
        ("Recettes base", "Moyenne"),
        ("Recettes réforme", "Moyenne"),
        ("Écart", "euros"),
        ("Écart", "%"),
    ]
]

In [ ]:
full[
    [
        ("Service", ""),
        ("Recettes base", "Moyenne"),
        ("Recettes réforme", "Moyenne"),
        ("Écart", "euros"),
        ("Écart", "%"),
    ]
]

In [ ]:
base_res_agent_df = pd.DataFrame(
    {
        "Service": [n for (n, df) in dfs],
        "Recettes base": [
            df[df.agent_ems].groupby("sample_id").prix.sum().median() for (n, df) in dfs
        ],
        "Quantité base": [
            df[df.agent_ems].groupby("sample_id").prix.count().median()
            for (n, df) in dfs
        ],
        "Recettes réforme": [
            df[df.agent_ems].groupby("sample_id").prix_r.sum().median()
            for (n, df) in dfs
        ],
    }
)
base_res_agent_df["Écart €"] = (
    base_res_agent_df["Recettes réforme"] - base_res_agent_df["Recettes base"]
)
base_res_agent_df["Écart %"] = (
    base_res_agent_df["Écart €"] / base_res_agent_df["Recettes base"]
)
res_agent_df = base_res_agent_df[base_res_agent_df["Recettes base"] != 0]
res_agent_df

In [ ]:
for n, df in dfs:
    df[df.agent_ems][["prix", "prix_r"]].sum()

In [ ]:
df = dfs[0][1]

In [ ]:
df

In [ ]:
df[df.agent_ems][["prix", "prix_r"]].sum()

In [ ]:
quantite_qf_caf = (
    df.groupby(["sample_id", "prix"]).quantité.sum().groupby("prix").median()
)
quantite_qf_caf

In [ ]:
quantite_qf_ems = (
    df.groupby(["sample_id", "prix_r"]).quantité.sum().groupby("prix_r").median()
)
quantite_qf_ems

In [ ]:
df

In [ ]:
df[
    (df.prix_r == 1) * (df.qf_fiscal != 0)
]  # .groupby('sample_id').quantité.sum().describe()

In [ ]:
if True:
    writer = pd.ExcelWriter("tables_sport.xlsx")
    df_qf = pd.DataFrame(
        {
            "quantité QF CAF": quantite_qf_caf,
            "quantité QF EMS": quantite_qf_ems,
        },
        index=quantite_qf_caf.index,
    )
    resume[
        [
            ("Recettes base", "Moyenne"),
            ("Recettes réforme", "Moyenne"),
            ("Écart", "euros"),
            ("Écart", "%"),
        ]
    ].to_excel(writer, sheet_name="recettes")
    full[
        [
            ("Service", ""),
            ("Recettes base", "Moyenne"),
            ("Recettes réforme", "Moyenne"),
            ("Écart", "euros"),
            ("Écart", "%"),
        ]
    ].to_excel(writer, sheet_name="recettes détaillées")
    df_qf.to_excel(writer, sheet_name="Prix")
    qf_nul = pd.DataFrame(
        {
            "quantité": [
                df[df.qf_fiscal == 0].groupby("sample_id").quantité.sum().median()
            ]
        }
    )
    qf_nul.to_excel(writer, sheet_name="QF_fiscal_nul")
    writer.close()

In [ ]:
for n, df in dfs:
    df["name"] = n

In [ ]:
full = pd.concat([d for (_, d) in dfs])

In [ ]:
df = full

In [ ]:
pd.pivot_table(df, index="prix", columns="name", values="quantité", aggfunc=sum)

In [ ]:
pd.pivot_table(df, index="prix_r", columns="name", values="quantité", aggfunc=sum)

In [ ]:
pd.pivot_table(
    df, index="prix_r", columns="prix", values="quantité", aggfunc=sum, fill_value=""
).to_excel("matrice_prix.xlsx")

In [ ]:
resume

In [ ]:
full

In [ ]:
pd.pivot_table(
    dfs[0][1],
    index=["prix", "sample_id"],
    columns="prix_r",
    values="quantité",
    aggfunc=sum,
).reset_index("prix").groupby(["prix"]).median()

In [ ]:
(recap, dfs) = get_results(base, 3, reform)
recap

In [ ]:
for v, df in dfs:
    print(v)
    display(df[~df.res])
    assert df[~df.res].shape[0] == 0

In [ ]:
prices = pd.Series(dtype="int64")
ddfs = []
for n, df in dfs:
    if n == "abo_annuel_ce" or n == "5 entrées ce" or n == "patinoire 5 entrees ce":
        continue
    print(n)
    sdf = df[df.sample_id == 0]
    bracket_limits = sdf.groupby("prix").quantité.sum().cumsum()
    limite_prix = bracket_limits[bracket_limits.index[-2]]
    print((bracket_limits, limite_prix))

    def determine_same_plein_tarif_qf(gdf):
        first_ranks = gdf.qf_fiscal.rank(method="first")
        max_ranks = gdf.qf_fiscal.rank(method="max")
        limit_rank = max_ranks[first_ranks == limite_prix].iloc[0]
        next_index = first_ranks[first_ranks == limit_rank + 1].index[0]
        next_row = gdf[gdf.index == next_index]
        return pd.DataFrame({n: [next_row.qf_fiscal.iloc[0]]})

    gdfs = df.groupby(by="sample_id")
    qf_plein_tarif = gdfs.apply(determine_same_plein_tarif_qf)
    ddfs.append(qf_plein_tarif)
    prices[n] = bracket_limits.index[-1]
    break

In [ ]:
tb = pd.pivot_table(
    dfs[0][1],
    index=["prix", "sample_id"],
    columns="prix_r",
    values="quantité",
    aggfunc=sum,
)
res_tb = tb.reset_index().groupby(["prix"]).describe()
res_tb

In [ ]:
tb.reset_index("prix").groupby(["prix"]).describe().transpose()

In [ ]:
tb.reset_index("prix").groupby(["prix"]).median()

In [ ]:
dfs[0][1].groupby(["prix", "sample_id"]).quantité.sum().reset_index("prix").groupby(
    "prix"
).median()

In [ ]:
dfs[0][1].groupby(["prix_r", "sample_id"]).quantité.sum().reset_index("prix_r").groupby(
    "prix_r"
).median()

In [ ]:
levels = (
    pd.DataFrame(data=[dd[dd.columns[0]] for dd in ddfs])
    .transpose()
    .median()
    .round()
    .astype("int")
)

In [ ]:
p_df = pd.DataFrame(data={"price": prices, "qf": levels})
p_df

In [ ]:
p_df[p_df.index == p_df.index[0]].price.iloc[0]

In [ ]:
def replace(local_parameters):
    for r in p_df.transpose():
        rdf = p_df[p_df.index == r]
        bf = fields[r]["bareme"]
        bb = bf(local_parameters)
        brackets = [
            new_bracket(0, 0),
            new_bracket(
                rdf.qf.iloc[0].astype("float"), rdf.price.iloc[0].astype("float")
            ),
        ]
        bb.brackets = brackets

    return local_parameters

In [ ]:
class customReform(Reform):
    def __init__(self, tbs):
        super().__init__(tbs)

    def apply(self):
        self.modify_parameters(modifier_function=replace)

In [ ]:
ld = levels.describe()
ld

In [ ]:
ld[ld.index == "50%"]

In [ ]:
rebase = CountryTaxBenefitSystem()
rebase.load_extension("openfisca_france_local")

In [ ]:
re_qf_fiscal_reform = QfFiscalReform(rebase)
re_grist_reform = gristSimulationReform(re_qf_fiscal_reform)
# re_max_reform = customReform(re_qf_fiscal_reform)
# re_statuts_reform = StatutReform(re_qf_fiscal_reform)
# re_statuts_reform = StatutReform(re_max_reform)
re_statuts_reform = StatutReform(re_qf_fiscal_reform)
re_ce_reform = CEReform(re_statuts_reform)
full_reform = re_ce_reform

In [ ]:
sss = get_results(rebase, 1, full_reform)

In [ ]:
sss[0]

In [ ]:
sss[0][("Recettes base", "Moyenne")].sum()

In [ ]:
sss[0][("Recettes réforme", "Moyenne")].sum()

In [ ]:
(sss[1][0][1].prix_r == sss[1][0][1].prix.max()).sum()

In [ ]:
(sss[1][0][1].prix == sss[1][0][1].prix.max()).sum()

In [ ]:
pd.pivot_table(
    sss[1][0][1],
    index=["prix", "sample_id"],
    columns="prix_r",
    values="quantité",
    aggfunc=sum,
).reset_index("prix").groupby(["prix"]).median()

In [ ]:
from scipy.optimize import fsolve, bisect

field = "entrée unitaire"


def solve_function(threshold):
    print(f"here {threshold}")

    def f_replace(local_parameters):
        bf = fields[field]["bareme"]
        bb = bf(local_parameters)
        brackets = [new_bracket(0, 0), new_bracket(threshold, 5)]
        bb.brackets = brackets

        return local_parameters

    class FCustomReform(Reform):
        def __init__(self, tbs):
            super().__init__(tbs)

        def apply(self):
            self.modify_parameters(modifier_function=f_replace)

    re_max_reform = FCustomReform(re_qf_fiscal_reform)
    re_statuts_reform = StatutReform(re_max_reform)
    # re_statuts_reform = StatutReform(re_grist_reform)
    re_ce_reform = CEReform(re_statuts_reform)
    full_reform = re_ce_reform
    sss = get_results(rebase, 1, full_reform, field)
    prix_max = sss[1][0][1].prix.max().sum()
    base = (sss[1][0][1].prix == sss[1][0][1].prix.max()).sum()
    result = (sss[1][0][1].prix_r == sss[1][0][1].prix.max()).sum()
    # print((prix_max, base, result))
    return result - base

In [ ]:
# bisect(solve_function, 680, 720, xtol=0.5)

In [ ]:
pd.pivot_table(
    sss[1][1][1],
    index=["prix", "sample_id"],
    columns="prix_r",
    values="quantité",
    aggfunc=sum,
).reset_index("prix").groupby(["prix"]).median()

In [ ]:
sss[0][("Recettes base", "Moyenne")].sum()

In [ ]:
sss[0][("Recettes réforme", "Moyenne")].sum()

In [ ]:
pd.pivot_table(
    sss[1][0][1],
    index=["prix", "sample_id"],
    columns="prix_r",
    values="quantité",
    aggfunc=sum,
).reset_index("prix").groupby(["prix"]).median()

In [ ]:
sss[1][0][1].groupby(["prix", "sample_id"]).quantité.sum().reset_index("prix").groupby(
    "prix"
).median()

In [ ]:
sss[1][0][1].groupby(["prix_r", "sample_id"]).quantité.sum().reset_index(
    "prix_r"
).groupby("prix_r").median()

In [ ]:
if False:
    df = get_df()

    get_total = lambda r: sum(r.prix * r.quantité)
    rows = []
    dfs = []
    for v in fields:
        if single and v != single:
            continue
        data, res = build_data(df, v, sample_count)
        openfisca_output_variable = fields[v]["openfisca_output_variable"]
        compute(tbs, data, res, openfisca_output_variable)

        count, value = extract(res, "prix")
        row = ["Sport", v]
        row.extend([count["mean"], count["count"]])
        row.extend(value)
        if reform:
            compute(reform, data, res, openfisca_output_variable, "_r")
            _, value = extract(res, "prix_r")
            row.extend(value)
        rows.append(row)
        dfs.append((v, res))

In [ ]:
if False:
    l = [
        ("entrée unitaire", 710.3125),
        ("10 entrées", 820.90625),
        ("abo_annuel", 926.96435546875),
        ("cycle", 901.6875),
        ("stage été", 769.875),
        ("stage vacances", 813.94140625),
        ("stage 5 séances", 318.0),
        ("patinoire entree unitaire", 776.1640625),
        ("patinoire 10 entrees", 1094.21875),
        ("entrée unitaire", 707.8125),
        ("10 entrées", 817.78125),
        ("abo_annuel", 917.484375),
        ("cycle", 808.875),
    ]

    for n, v in l:
        LimitContainer.set(n, v)

In [ ]:
import warnings

In [ ]:
from scipy.optimize import fsolve, bisect
import warnings

df = get_df()
sample_count = 1

In [ ]:
# for i in range(5):
#    for field in fields:
while False:
    r = LimitContainer.raw()
    l = [(i, len(r[i])) for i in r if i != "stage 5 séances"]
    l.sort(key=lambda i: i[1])
    for field in [l[0][0]]:
        bf = fields[field]["bareme"]
        if not bf:
            continue
        print(field)

        data, res = build_data(df, field, sample_count)
        openfisca_output_variable = fields[field]["openfisca_output_variable"]
        compute(rebase, data, res, openfisca_output_variable)

        def solve_function_in(threshold):
            # print(threshold)
            def f_replace(local_parameters):
                bf = fields[field]["bareme"]
                bb = bf(local_parameters)
                brackets = [
                    new_bracket(0, 0),
                    new_bracket(threshold, res.prix.max().astype("float")),
                ]
                bb.brackets = brackets

                return local_parameters

            class FCustomReform(Reform):
                def __init__(self, tbs):
                    super().__init__(tbs)

                def apply(self):
                    self.modify_parameters(modifier_function=f_replace)

            re_max_reform = FCustomReform(re_qf_fiscal_reform)
            re_statuts_reform = StatutReform(re_max_reform)
            # re_statuts_reform = StatutReform(re_grist_reform)
            re_ce_reform = CEReform(re_statuts_reform)
            full_reform = re_ce_reform
            compute(full_reform, data, res, openfisca_output_variable, "_r")
            prix_max = res.prix.max().sum()
            base = (res.prix == res.prix.max()).sum()
            result = (res.prix_r == res.prix.max()).sum()
            # print((prix_max, base, result))
            return result - base

        rr = None
        bands = [0, 20, 50, 100, 500, 1000]  # , 2000, 5000]
        # bands = [2000]
        previous = LimitContainer.get(field)
        error = None
        if len(previous) == 0:
            n = [500, 1000]
        else:
            std = max(5, np.std(previous))
            mid = np.median(previous)
            n = [mid - 2 * std, mid + 2 * std]
        for b in bands:
            interval = [n[0] - b, n[1] + b]
            print(interval)
            try:
                rr = bisect(solve_function_in, interval[0], interval[1], xtol=0.5)
                break
            except ValueError as e:
                error = e
                pass
        if rr == None:
            warnings.warn("What a mess!")
            # raise ValueError('What a mess!')

        print((field, rr))
        LimitContainer.set(field, rr)

```python
"What a mess!"
"stage 5 séances"
[-1716.933966519941, 2322.933966519941]
[-2064.4076545177413, 2670.4076545177413]
[-2064.4076545177413, 2670.4076545177413]
[-2080.8878586135247, 2691.8878586135247]
```